In [1]:
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
import seaborn as sns
import numpy as np


from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, make_pipeline
import pandas as pd

#https://machinelearningmastery.com/columntransformer-for-numerical-and-categorical-data/

In [2]:
df= pd.read_csv('train.csv', index_col=0, parse_dates=True)

In [3]:
df['weekday']=df.index.day_of_week #good predictor probably (recurring things happening per day of week)
df['time']=df.index.hour

In [4]:
#df for cas
df_reg = df.drop(['count', 'casual', 'atemp'], axis=1) #also drop atemp, multico 
df_reg

,season,holiday,workingday,weather,temp,humidity,windspeed,registered,weekday,time
datetime,,,,,,,,,,
2011-01-01 00:00:00,1,0,0,1,9.84,81,0.0000,13,5,0
2011-01-01 01:00:00,1,0,0,1,9.02,80,0.0000,32,5,1
2011-01-01 02:00:00,1,0,0,1,9.02,80,0.0000,27,5,2
2011-01-01 03:00:00,1,0,0,1,9.84,75,0.0000,10,5,3
2011-01-01 04:00:00,1,0,0,1,9.84,75,0.0000,1,5,4
...,...,...,...,...,...,...,...,...,...,...
2012-12-19 19:00:00,4,0,1,1,15.58,50,26.0027,329,2,19
2012-12-19 20:00:00,4,0,1,1,14.76,57,15.0013,231,2,20
2012-12-19 21:00:00,4,0,1,1,13.94,61,15.0013,164,2,21


In [5]:
#interaction features here? humidity, holidaz and weekday?


In [6]:
X =df_reg[['season', 'holiday', 'workingday', 'weather', 'temp', 'humidity', 'windspeed', 'weekday', 'time']]
y= df_reg['registered']
y

datetime
2011-01-01 00:00:00     13
2011-01-01 01:00:00     32
2011-01-01 02:00:00     27
2011-01-01 03:00:00     10
2011-01-01 04:00:00      1
                      ... 
2012-12-19 19:00:00    329
2012-12-19 20:00:00    231
2012-12-19 21:00:00    164
2012-12-19 22:00:00    117
2012-12-19 23:00:00     84
Name: registered, Length: 10886, dtype: int64

In [7]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, make_pipeline
#test, train, split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state = 10)
X_train.shape, X_test.shape

((8164, 9), (2722, 9))

In [8]:
X_train

,season,holiday,workingday,weather,temp,humidity,windspeed,weekday,time
datetime,,,,,,,,,
2012-10-12 09:00:00,4,0,1,1,20.50,39,19.9995,4,9
2011-09-09 23:00:00,3,0,1,1,25.42,94,6.0032,4,23
2011-06-08 18:00:00,2,0,1,1,36.90,37,15.0013,2,18
2011-10-06 01:00:00,4,0,1,1,19.68,63,12.9980,3,1
2011-10-10 12:00:00,4,1,0,2,28.70,48,6.0032,0,12
...,...,...,...,...,...,...,...,...,...
2012-09-13 21:00:00,3,0,1,1,25.42,65,7.0015,3,21
2012-05-03 04:00:00,2,0,1,2,18.86,88,16.9979,3,4
2011-04-01 21:00:00,2,0,1,1,13.12,53,8.9981,4,21


In [9]:
 #(binning for time)
pipe1 = Pipeline([
    ('kbins', KBinsDiscretizer(n_bins=6, encode='onehot-dense', strategy='uniform'))
])

In [11]:
#humidity , windspeed, temp

pipe2 = Pipeline([('scaler', StandardScaler())])


In [12]:
# feature engineering for categorical
pipe3 = Pipeline([('ohe', OneHotEncoder(handle_unknown="ignore"))])

In [13]:
trans = ColumnTransformer([
    ('pipe1', pipe1, ['time']), 
    ('pipe2', pipe2, ['humidity', 'windspeed','temp']), 
    ('pipe3', pipe3,['workingday', 'holiday', 'season', 'weekday'])

]) 

In [14]:
from sklearn.linear_model import LinearRegression

In [15]:
model0 =LinearRegression()

In [16]:
pipe5 = Pipeline(steps=[
    ('trans', trans), 
    ('model0', LinearRegression())
     ])

In [17]:
pipe5.fit(X_train, y_train)
# make predictions

Pipeline(steps=[('trans',
                 ColumnTransformer(transformers=[('pipe1',
                                                  Pipeline(steps=[('kbins',
                                                                   KBinsDiscretizer(encode='onehot-dense',
                                                                                    n_bins=6,
                                                                                    strategy='uniform'))]),
                                                  ['time']),
                                                 ('pipe2',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler())]),
                                                  ['humidity', 'windspeed',
                                                   'temp']),
                                                 ('pipe3',
                                                 

In [18]:
# make predictions
yhat = pipe5.predict(X_test)

In [19]:
from sklearn import metrics

In [20]:
r2_test = metrics.r2_score(y_test, yhat)

r2_test 
#much much worse than casual, only 46 as opposed to 56?? Probably need interaction features... 

0.46831219055184736

In [ ]:
#grid search to see which parameters important 

#https://towardsdatascience.com/easy-way-to-create-an-algorithm-chains-pipelines-with-grid-search-columntransformer-feature-24feb3d6aec5


In [ ]:
#print("model score: %.3f" % pipeline.score(X_test, y_test))
# get predictions from the pipeline
#print(pipeline.predict(X_test))

# get prediction probabilities from the pipeline 
#print(pipeline.predict_proba(X_test)[:, 1])

In [21]:
#linreg without FE
#m = LinearRegression()
#m.fit(X_train,y_train)
#m.score(X_test, y_test)
# m.predict(X)
#y_pred = m.predict(X)
#df['y_pred'] = y_pred